In [ ]:
import asyncio
import logging
import sys

from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()
await asyncio.sleep(10)

In [ ]:
atcs = ATCS(domain)
latiss = LATISS(domain)

await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
date = Time.now()
group_id = f'CALSET_{date.tai.strftime("%Y%m%d_%H%M")}'

In [ ]:
await latiss.take_bias(nbias=10, group_id=group_id)

In [ ]:
await latiss.take_darks(exptime=5.0, ndarks=10, group_id=group_id)

In [ ]:
#await atcs.prepare_for_flatfield()
# The following approximates the above because some things don't work well
await atcs.open_m1_cover()
await atcs.point_azel(target_name="FlatField position",
                      az=atcs.tel_flat_az,
                      el=atcs.tel_flat_el,
                      wait_dome=False)
await atcs.slew_dome_to(atcs.dome_flat_az)

In [ ]:
await latiss.take_flats(exptime=2.0, nflats=10, group_id=group_id)